In [1]:
## Text to embeddings
## embeddings available for openai and ollama 

In [1]:
from mb_rag.rag.embeddings import embedding_generator

In [2]:
embedding_generator?

Init signature:
embedding_generator(
    model: str = 'openai',
    model_type: str = 'text-embedding-3-small',
    vector_store_type: str = 'chroma',
    collection_name: str = 'test',
    logger=None,
    model_kwargs: dict = None,
    vector_store_kwargs: dict = None,
) -> None
Docstring:     
Main class for generating embeddings and managing RAG operations.

This class provides comprehensive functionality for generating embeddings,
managing vector stores, handling retrievers, and managing conversations.

Args:
    model (str): Model provider name (default: 'openai')
    model_type (str): Model type/identifier (default: 'text-embedding-3-small')
    vector_store_type (str): Type of vector store (default: 'chroma')
    collection_name (str): Name of the collection (default: 'test')
    logger: Optional logger instance
    model_kwargs (dict): Additional arguments for model initialization
    vector_store_kwargs (dict): Additional arguments for vector store initialization

Example:
  

In [3]:
em_gen = embedding_generator(model="openai",model_type="text-embedding-3-small",vector_store_type="chroma")

In [ ]:
em_gen.generate_text_embeddings(text_data_path=['./examples/book_data.txt'],chunk_size=500,chunk_overlap=20,
                                folder_save_path='./examples/book_data_embeddings',replace_existing=True)

/home/malav/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-07 05:50:32.280912: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733550632.296813 1618158 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733550632.301774 1618158 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 05:50:32.319535: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operation

Text data splitted into 1250 chunks


In [5]:
em_loading = em_gen.load_embeddings(embeddings_folder_path='./examples/book_data_embeddings')

In [6]:
em_loading

In [12]:
em_retriever = em_gen.load_retriever(embeddings_folder_path='./examples/book_data_embeddings',search_params=[{"k": 2,"score_threshold": 0.1}])

In [13]:
ans = em_retriever.invoke("what is the Book about?")
# try:
    # ans = em_retriever.invoke("what is the Book about?")
# except AttributeError:
    # If ChatOpenAI doesn't have embed_query, use the embedding model directly
# query_embedding = em_gen.model.embed_documents(["what is the Book about?"])[0]
# ans = em_retriever.get_relevant_documents(query_embedding)

No relevant docs were retrieved using the relevance score threshold 0.1


In [14]:
for i,k in enumerate(ans):
    print(f"Document {i}:\n{k.page_content}\n")
    print(f"Source: {k.metadata['source']}\n")

In [15]:
## testing history aware rag chain / retriever
# Generate RAG chain
rag_chain = em_gen.generate_rag_chain(retriever=em_retriever)


In [16]:
# Test the RAG chain
test_questions = [
    "What is the main theme of the book?",
    "Who are the main characters?",
    "Can you summarize the plot?",
    "What is the setting of the story?",
    "What genre does this book belong to?"
]

print("Testing RAG chain with multiple questions:\n")
for question in test_questions:
    print(f"Question: {question}")
    response = em_gen.conversation_chain(question, rag_chain,file='./examples/book_data_rag_chain.txt')
    print(f"Response: {response}\n")

Testing RAG chain with multiple questions:

Question: What is the main theme of the book?


No relevant docs were retrieved using the relevance score threshold 0.1


Response: The main theme of the book "Adventures of Huckleberry Finn" is the conflict between civilization and freedom. The book explores the struggles of the protagonist, Huck, as he seeks to escape societal expectations and constraints. It also addresses issues of race and identity in pre-Civil War America.
Saved file : ./examples/book_data_rag_chain.txt
Response: ["content='You : What is the main theme of the book?'", 'content=\'The main theme of "Adventures of Huckleberry Finn" is the conflict between civilization and freedom. It explores the struggles of the protagonist, Huck, as he seeks to escape societal expectations and constraints, while also addressing issues of race and identity in pre-Civil War America.\'', "content='You : Who are the main characters?'", 'content=\'The main characters in "Adventures of Huckleberry Finn" are Huck Finn and Tom Sawyer. Huck Finn is the protagonist, and Tom Sawyer is his friend and comrade in various adventures.\'', "content='You : Can you sum

No relevant docs were retrieved using the relevance score threshold 0.1


Response: The main characters in "Adventures of Huckleberry Finn" are Huck Finn and Tom Sawyer. Huck Finn is the protagonist of the story, and Tom Sawyer is his friend who accompanies him in various adventures.
Saved file : ./examples/book_data_rag_chain.txt
Response: ["content='You : What is the main theme of the book?'", 'content=\'The main theme of "Adventures of Huckleberry Finn" is the conflict between civilization and freedom. It explores the struggles of the protagonist, Huck, as he seeks to escape societal expectations and constraints, while also addressing issues of race and identity in pre-Civil War America.\'', "content='You : Who are the main characters?'", 'content=\'The main characters in "Adventures of Huckleberry Finn" are Huck Finn and Tom Sawyer. Huck Finn is the protagonist, and Tom Sawyer is his friend and comrade in various adventures.\'', "content='You : Can you summarize the plot?'", 'content=\'"Adventures of Huckleberry Finn" follows Huck Finn, a young boy who f

...

Response: content='"Adventures of Huckleberry Finn" is about a young boy named Huck Finn who fakes his own death to escape his abusive father. He joins forces with Jim, a runaway slave, and they journey down the Mississippi River on a raft. Throughout their journey, Huck struggles with his conscience and societal values, eventually deciding to aid Jim in his quest for freedom.'
Saved file : ./examples/book_data_rag_chain.txt
Response: ["content='You : What is the main theme of the book?'", 'content=\'The main theme of "Adventures of Huckleberry Finn" is the conflict between civilization and freedom. It explores the struggles of the protagonist, Huck, as he seeks to escape societal expectations and constraints, while also addressing issues of race and identity in pre-Civil War America.\'', "content='You : Who are the main characters?'", 'content=\'The main characters in "Adventures of Huckleberry Finn" are Huck Finn and Tom Sawyer. Huck Finn is the protagonist, and Tom Sawyer is his fri

No relevant docs were retrieved using the relevance score threshold 0.1


Response: content='The story of "Adventures of Huckleberry Finn" is set along the Mississippi River, primarily in the fictional town of St. Petersburg, Missouri, during the mid-19th century. The river and its surrounding areas play a pivotal role in the adventures of Huck Finn and Jim.'
Saved file : ./examples/book_data_rag_chain.txt
Response: ["content='You : What is the main theme of the book?'", 'content=\'The main theme of "Adventures of Huckleberry Finn" is the conflict between civilization and freedom. It explores the struggles of the protagonist, Huck, as he seeks to escape societal expectations and constraints, while also addressing issues of race and identity in pre-Civil War America.\'', "content='You : Who are the main characters?'", 'content=\'The main characters in "Adventures of Huckleberry Finn" are Huck Finn and Tom Sawyer. Huck Finn is the protagonist, and Tom Sawyer is his friend and comrade in various adventures.\'', "content='You : Can you summarize the plot?'", 'co

No relevant docs were retrieved using the relevance score threshold 0.1


Response: content='"Adventures of Huckleberry Finn" by Mark Twain is an adventure fiction. It is also regarded as a classic of American literature with elements of satire and social criticism.'
Saved file : ./examples/book_data_rag_chain.txt
Response: ["content='You : What is the main theme of the book?'", 'content=\'The main theme of "Adventures of Huckleberry Finn" is the conflict between civilization and freedom. It explores the struggles of the protagonist, Huck, as he seeks to escape societal expectations and constraints, while also addressing issues of race and identity in pre-Civil War America.\'', "content='You : Who are the main characters?'", 'content=\'The main characters in "Adventures of Huckleberry Finn" are Huck Finn and Tom Sawyer. Huck Finn is the protagonist, and Tom Sawyer is his friend and comrade in various adventures.\'', "content='You : Can you summarize the plot?'", 'content=\'"Adventures of Huckleberry Finn" follows Huck Finn, a young boy who fakes his own deat

In [17]:
# Test query_embeddings function
print("Testing query_embeddings function:\n")
query = "What are the key themes explored in the book?"
result = em_gen.query_embeddings(query)
print(f"Query: {query}")
print(f"Result: {result}\n")

# Test get_relevant_documents function
print("Testing get_relevant_documents function:\n")
query = "Describe the main conflict in the story."
docs = em_gen.get_relevant_documents(query)
print(f"Query: {query}")
print("Relevant documents:")
for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content[:100]}...")  # Print first 100 characters
    print(f"Source: {doc.metadata['source']}\n")


Testing query_embeddings function:



No relevant docs were retrieved using the relevance score threshold 0.1
/home/malav/Desktop/mb_packages/mb_rag/mb_rag/rag/embeddings.py:546: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return retriever.get_relevant_documents(query)


Query: What are the key themes explored in the book?
Result: []

Testing get_relevant_documents function:



No relevant docs were retrieved using the relevance score threshold 0.1


Query: Describe the main conflict in the story.
Relevant documents:


In [18]:
em_gen.load_conversation('./examples/book_data_rag_chain.txt',list_type=True)

["content='You : What is the main theme of the book?'",
 'content=\'The main theme of "Adventures of Huckleberry Finn" is the conflict between civilization and freedom. It explores the struggles of the protagonist, Huck, as he seeks to escape societal expectations and constraints, while also addressing issues of race and identity in pre-Civil War America.\'',
 "content='You : Who are the main characters?'",
 'content=\'The main characters in "Adventures of Huckleberry Finn" are Huck Finn and Tom Sawyer. Huck Finn is the protagonist, and Tom Sawyer is his friend and comrade in various adventures.\'',
 "content='You : Can you summarize the plot?'",
 'content=\'"Adventures of Huckleberry Finn" follows Huck Finn, a young boy who fakes his own death to escape his abusive father. He teams up with Jim, a runaway slave, and they journey down the Mississippi River on a raft, encountering various adventures and characters. Throughout their journey, Huck grapples with his conscience and societal 

In [19]:
## testing firecrawl
em_gen.firecrawl_web(website="https://www.github.com/bigmb",mode="scrape",file_to_save='./examples/firecrawl_embeddings')

[2024-12-07 05:57:35 - langchain_text_splitters.base:122 - WARNING] Created a chunk of size 11660, which is longer than the specified 1000
[2024-12-07 05:57:35 - langchain_text_splitters.base:122 - WARNING] Created a chunk of size 1495, which is longer than the specified 1000
[2024-12-07 05:57:35 - chromadb.db.impl.sqlite:111 - WARNING] ⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.



--- Document Chunks Information ---
Number of document chunks: 16
Sample chunk:
You signed in with another tab or window. Reload to refresh your session.You signed out in another tab or window. Reload to refresh your session.You switched accounts on another tab or window. Reload to refresh your session.Dismiss alert

{{ message }}

![@bigmb](https://avatars.githubusercontent.com/u/14040051?s=64&v=4)**bigmb**[Follow](/login?return_to=https%3A%2F%2Fgithub.com%2Fbigmb)

More

- [Overview](/bigmb)
- [Repositories](/bigmb?tab=repositories)
- [Projects](/bigmb?tab=projects)
- [Packages](/bigmb?tab=packages)
- [Stars](/bigmb?tab=stars)

![@bigmb](https://avatars.githubusercontent.com/u/14040051?s=64&v=4)
**bigmb**
[Follow](/login?return_to=https%3A%2F%2Fgithub.com%2Fbigmb)

[![View bigmb's full-sized avatar](https://avatars.githubusercontent.com/u/14040051?v=4)](https://avatars.githubusercontent.com/u/14040051?v=4)

![:octocat:](https://github.githubassets.com/images/icons/emoji/octocat.png)